In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    TopKLogitsWarper,
    TemperatureLogitsWarper,
    StoppingCriteriaList,
    MaxLengthCriteria,
    XGLMForCausalLM,
)
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-564M")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-564M")


In [3]:
model.cuda()


XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 1024, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_pr

In [23]:
import pandas as pd 

df = pd.read_csv('./data/sample_train.csv')

In [24]:
for i in range(len(df)):
    dirty = df['text'][i]
    clean = df['clean'][i]
    input_prompt = f'นี่คือประโยคที่ไม่สุภาพ: \"{dirty}\" \nนี่คือประโยคที่สุภาพ: \"{clean}\"'
    input_ids = tokenizer(input_prompt, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
    logits_processor = LogitsProcessorList(
        [
            MinLengthLogitsProcessor(
                15, eos_token_id=model.generation_config.eos_token_id),
        ]
    )
    torch.manual_seed(0)
    output = model.generate(input_ids=input_ids, top_k=50, temperature=0.7, max_length=256, early_stopping=True, no_repeat_ngram_size=2,
                            # min_length=len(input_ids),
                            # num_beams=5, no_repeat_ngram_size=2, early_stopping=True, max_new_tokens=100,
                            logits_processor=logits_processor)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    torch.cuda.empty_cache()
    print(f"{generated_text}")
    # print(f"dirty: {dirty}, clean: {clean}")




นี่คือประโยคที่ไม่สุภาพ: "ก็เพราะมันจะมาทําลายธุรกิจถูกกฎหมายของประเทศที่ทํางินมหาศาลในแต่ละปีไงล่ะจึงต้องตัดไฟเสียแต่ต้นลม" นี่คือประโยคที่สุภาพ: "เพราะธุรกิจที่ผิดกฏหมายเหล่านั้นจะทําลายธุรกิจที่ถูกกฏหมายในประเทศ จึงต้องตัดไฟเสียเเต่ต้นลม"
นี่คือประโยคที่ไม่สุภาพ: "อยากกิน ไปมั้ย" นี่คือประโยคที่สุภาพ: "ไปกินกันไหม"
นี่คือประโยคที่ไม่สุภาพ: "เหยดเข้ มีชีสสส จัดไปป" นี่คือประโยคที่สุภาพ: "มีชีสด้วย ไปกินกันไหม"
นี่คือประโยคที่ไม่สุภาพ: "ดูหญิงแย้สอนใส่ผ้าอนามัยโคตรฮาแม่ง55555555555 แบบกลางคืน40เซนนี่ผ้าอนามัยหรือกล่องดินสอ" นี่คือประโยคที่สุภาพ: "ดูหญิงแย้สอนใส่ผ้าอนามัยตลกมาก เขาบอกว่าเเบบกลางคืน ยาว 40 เซนติเมตร นั่นผ้าอนามัยหรือกล่องดินสอคะ"
นี่คือประโยคที่ไม่สุภาพ: "ค่าเช่าชุดก็500 ล่ะ แพงกว่าปกติอีก555" นี่คือประโยคที่สุภาพ: "ค่าเช่าชุดเสื้อผ้า 500 บาท สูงเกินไปจริงๆ ถ้าเปรียบเทียบกับราคาปกติมากขึ้น ขอสอบถามได้ไหมคะว่าเพราะสาเหตุใด"
นี่คือประโยคที่ไม่สุภาพ: "อยากอ่ดิ" นี่คือประโยคที่สุภาพ: "อยากใช่ไหม"
นี่คือประโยคที่ไม่สุภาพ: "เพียวๆๆเลยแอด." นี่คือประโยคที่สุภาพ: "กินปล่าวๆ ไปเ

In [32]:
input_prompt = f'นี่คือประโยคที่ไม่สุภาพ \"สรุปแดกลีโอต่อปาย\" \nนี่คือประโยคที่สุภาพ สรุปแล้วว่า \"เลือกดื่มลีโอต่อไป\" \
                นี่คือประโยคที่ไม่สุภาพ \"กูคนเดียวป้ะที่ใช้mamonde 23ละออกชมพูแปร๋นอยู่คนเดียว ทําไมมันไม่ตุ่นเหมือนชาวบ้านเขาบ้างงงง!!!!!!\" \nนี่คือประโยคที่สุภาพ '
input_ids = tokenizer(input_prompt, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
output = model.generate(input_ids=input_ids, 
                        top_k=50, 
                        temperature=0.7, 
                        max_length=256, 
                        early_stopping=True, 
                        no_repeat_ngram_size=2,
                        logits_processor=logits_processor)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
torch.cuda.empty_cache()
print(generated_text)

นี่คือประโยคที่ไม่สุภาพ "สรุปแดกลีโอต่อปาย" นี่คือประโยคที่สุภาพ สรุปแล้วว่า "เลือกดื่มลีโอต่อไป" นี่คือประโยคที่ไม่สุภาพ "กูคนเดียวป้ะที่ใช้mamonde 23ละออกชมพูแปร๋นอยู่คนเดียว ทําไมมันไม่ตุ่นเหมือนชาวบ้านเขาบ้างงงง!!!!!!" นี่คือประโยคที่สุภาพ อธิบายว่า กูไม่ได้ใช้ "แม่" นะ มันเป็น "ลูก"
